In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import plotly.express as px
import numpy as np
import pandas as pd

In [2]:
all_buildings_df = pd.read_parquet("../data/processed/all_buildings.parquet")

all_buildings_df.replace(0, np.nan, inplace=True)

all_buildings_df.drop(columns=['10.6-PF1'], inplace=True)

all_buildings_df = all_buildings_df.asfreq("h")

In [ ]:
all_buildings_diff_df = all_buildings_df.ffill().diff()[~all_buildings_df.isna()]

all_buildings_diff_df = all_buildings_diff_df[all_buildings_diff_df > 0]
all_buildings_diff_df = all_buildings_diff_df[all_buildings_diff_df < 2000]

# convert all to kWh
for col in [x for x in all_buildings_diff_df if "TF" in x]:
    all_buildings_diff_df[col] = all_buildings_diff_df[col] * 277.78

for col in [x for x in all_buildings_diff_df if "PF" in x]:
    all_buildings_diff_df[col] = all_buildings_diff_df[col] * 10.694
    
all_buildings_cumsum_df = all_buildings_diff_df.cumsum()

all_buildings_cumsum_df

In [ ]:
px.scatter(all_buildings_diff_df)

In [ ]:
px.scatter(all_buildings_cumsum_df)

In [7]:
all_buildings_diff_df.to_parquet("../data/processed/all_buildings_cleaned_diff_kwh.parquet")

all_buildings_cumsum_df.to_parquet("../data/processed/all_buildings_cleaned_cumsum_kwh.parquet")

In [7]:
all_buildings_year_max_df = all_buildings_cumsum_df.resample("YE").max()

all_buildings_year_min_df = all_buildings_cumsum_df.resample("YE").min()

all_buildings_year_dif_df = all_buildings_year_max_df - all_buildings_year_min_df

all_buildings_year_dif_df.index = [2021, 2022, 2023, 2024]

all_buildings_year_dif_df.drop(2024, inplace=True)


In [ ]:

all_buildings_year_dif_df

In [9]:
all_buildings_year_dif_df.to_parquet("../data/processed/all_buildings_yearly_kwh.parquet")

In [ ]:
px.bar(all_buildings_year_dif_df,
       barmode="group",
       title="Yearly primary energy demand [kWh]")

In [ ]:
all_buildings_year_dif_df